![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Instalación de dependencias y configuración del entorno en GPU

In [ ]:
#!pip install tensorflow-gpu==2.0.0.alpha0

In [ ]:
!pip install tqdm

### Descargar el dataset de Perros vs Gatos

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2021-01-11 17:49:29--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 74.125.142.128, 2607:f8b0:400e:c08::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   171MB/s    in 0.4s    

2021-01-11 17:49:29 (171 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Pre procesado del data set

### Importar las dependencias del proyecto

In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.4.0'

### Descomprimir el dataset de Perros vs Gatos

In [ ]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [ ]:
with zipfile.ZipFile(file=dataset_path, mode="r") as zip_object:
  zip_object.extractall("./data")

### Configurar las rutas al dataset

In [ ]:
dataset_path_new = "./data/cats_and_dogs_filtered/"

In [ ]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construir el Modelo

### Cargar un modelo pre entrenado (MobileNetV2)

In [ ]:
IMG_SHAPE = (128, 128, 3)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
#base_model = tf.keras.applications.MobileNetV2( weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

### Congelar el modelo base

In [ ]:
base_model.trainable = False

### Definir la cabecera personalizada para nuestra red neuronal

In [ ]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [ ]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Definir el modelo

In [ ]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

### Compilar el modelo

In [ ]:
optimizer=tf.keras.optimizers.RMSprop(lr=0.0001)
#optimizer=tf.keras.optimizers.SGD( learning_rate=0.01)

In [ ]:
model.compile(optimizer=optimizer ,loss="binary_crossentropy", metrics=["accuracy"])

### Crear generadores de datos

Redimensionar imágenes

    Las grandes arquitecturas pre-entrenadas solamente soportan cierto tipo de tamaños de imágenes.
    
Por ejemplo: MobileNet (la arquitectura que nosotros usamos) soporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [ ]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [ ]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Entrenar el modelo 

In [ ]:
model.fit_generator(train_generator, epochs=8, validation_data=valid_generator)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/8
16/16 [==============================] - 16s 540ms/step - loss: 0.7997 - accuracy: 0.4793 - val_loss: 0.6834 - val_accuracy: 0.5970
Epoch 2/8
16/16 [==============================] - 7s 463ms/step - loss: 0.6672 - accuracy: 0.6272 - val_loss: 0.5923 - val_accuracy: 0.6940
Epoch 3/8
16/16 [==============================] - 7s 464ms/step - loss: 0.5658 - accuracy: 0.7122 - val_loss: 0.5176 - val_accuracy: 0.7700
Epoch 4/8
16/16 [==============================] - 7s 469ms/step - loss: 0.5063 - accuracy: 0.7681 - val_loss: 0.4555 - val_accuracy: 0.8210
Epoch 5/8
16/16 [==============================] - 7s 463ms/step - loss: 0.4482 - accuracy: 0.8128 - val_loss: 0.4040 - val_accuracy: 0.8500
Epoch 6/8
16/16 [==============================] - 7s 471ms/step - loss: 0.3932 - accuracy: 0.8581 - val_loss: 0.3600 - val_accuracy: 0.8770
Epoch 7/8
16/16 [==============================] - 7s 467ms/step - loss: 0.3476 - accuracy: 0.8831 - val_loss: 0.3235 - val_accuracy: 0.9020
Epoch 8/8
16

In [ ]:
optimizer.get_weights()

[128, array([[0.00035218],
        [0.00055212],
        [0.00123953],
        ...,
        [0.00306381],
        [0.00038528],
        [0.00189844]], dtype=float32), array([0.00085843], dtype=float32)]

In [ ]:
optimizer.variables()

[<tf.Variable 'RMSprop/iter:0' shape=() dtype=int64, numpy=128>,
 <tf.Variable 'RMSprop/dense/kernel/rms:0' shape=(1280, 1) dtype=float32, numpy=
 array([[0.00035218],
        [0.00055212],
        [0.00123953],
        ...,
        [0.00306381],
        [0.00038528],
        [0.00189844]], dtype=float32)>,
 <tf.Variable 'RMSprop/dense/bias/rms:0' shape=(1,) dtype=float32, numpy=array([0.00085843], dtype=float32)>]

### Evaluar el modelo de aprendizaje por transferencia

In [ ]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

8/8 [==============================] - 2s 302ms/step - loss: 0.2931 - accuracy: 0.9070


In [ ]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9070000052452087


## Puesta a punto de parámetros


Un par de cosas:

- NUNCA HAY QUE USAR la puesta a punto (fine tuning) de parámetros en toda la red neuronal: con algunas de las capas superiores (las finales) es más que suficiente suficiente. En la mayoría de casos, son las más especializadas. El objetivo del fine tuning es adaptar esa parte específica de la red neuronal para nuestro nuevo dataset específico.
- Empezar con la puesta a punto DESPUÉS de haber finalizado la fase de aprendizaje por transferencia. Si intentamos hacer el Fine tuning inmediatamente, los gradientes serán muy diferentes entre nuestra cabecera personalizada de la red neuronal y las nuevas capas no congeladas del modelo base.

### Descongelar unas cuantas capas superiores del modelo

In [ ]:
base_model.trainable = True

In [ ]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 154


In [ ]:
fine_tune_at = 100

In [ ]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compilar el modelo para la puesta a punto

In [ ]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Puesta a punto

In [ ]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
16/16 [==============================] - 12s 526ms/step - loss: 0.2631 - accuracy: 0.8862 - val_loss: 0.1148 - val_accuracy: 0.9570
Epoch 2/5
16/16 [==============================] - 7s 472ms/step - loss: 0.0299 - accuracy: 0.9971 - val_loss: 0.1147 - val_accuracy: 0.9610
Epoch 3/5
16/16 [==============================] - 7s 476ms/step - loss: 0.0089 - accuracy: 1.0000 - val_loss: 0.1620 - val_accuracy: 0.9530
Epoch 4/5
16/16 [==============================] - 8s 482ms/step - loss: 0.0042 - accuracy: 1.0000 - val_loss: 0.2046 - val_accuracy: 0.9510
Epoch 5/5
16/16 [==============================] - 7s 482ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.1867 - val_accuracy: 0.9570


In [ ]:
len(optimizer.get_weights())

57

### Evaluar el modelo re calibrado

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [ ]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9570000171661377
